In [14]:
import os
# !{os.sys.executable} -m pip install gymnasium
# !{os.sys.executable} -m pip install cmake 
# !{os.sys.executable} -m pip install ale-py
# !{os.sys.executable} -m pip install atari-py
# !{os.sys.executable} -m pip install Pillow
# !{os.sys.executable} -m pip install ipython
# !{os.sys.executable} -m pip install pygame
# !{os.sys.executable} -m pip install torchsummary
# !{os.sys.executable} -m pip install tensorboardX
# !{os.sys.executable} -m pip install "gymnasium[atari, accept-rom-licesnse]"
# !{os.sys.executable} -m pip install "autorom[accept-rom-license]"
# !{os.sys.executable} -m pip install torch
# !{os.sys.executable} -m pip install torchvision


In [15]:
# Import useful packages

import os
import sys
import gymnasium as gym
import random
import numpy as np

import torch
import torch.optim as optim
import torch.nn.functional as F
from torchinfo import summary

from DQN_Model import DQN
from DQN_Model_improved import DQN_improved
from tensorboardX import SummaryWriter

from collections import namedtuple, deque

In [16]:
env_name = 'ALE/Breakout-v5'
goal_score = 200
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device", device)

device cuda


In [17]:
import pygame
from pygame.locals import QUIT

# Initialize pygame
pygame.init()

def play_game_with_model(model_name, load_best = False):
    # Initialize the environment
    env = gym.make(env_name, render_mode="human")
    state = env.reset()[0]
    # state, _, _, _,_ = env.step(1)
    done = False
    
    # Load the model
    state_size = env.observation_space.low.size
    action_size = env.action_space.n
    loaded_online_net = DQN(state_size, action_size).to(device)
    if( not load_best):
        loaded_online_net.load_state_dict(torch.load(f'saved_models/breakout_DQN/{model_name}/online_net.pth', map_location = device))
    else:
        loaded_online_net.load_state_dict(torch.load(f'saved_models/breakout_DQN/{model_name}/best_online_net.pth', map_location = device))
    
    loaded_online_net.eval()  # Set the model to evaluation mode


    # Initialize the display
    # screen = pygame.display.set_mode((600, 400))
    # pygame.display.set_caption("Breakout with DQN")
    # clock = pygame.time.Clock()
    score = 0
    while not done:
        # for event in pygame.event.get():
        #     if event.type == QUIT:
        #         print("Quit")
        #         pygame.quit()
        #         done = True
        #         return

        # Preprocess the state
        # print("A")
        state_tensor = torch.Tensor(state).to(device)
        state_tensor = torch.flatten(state_tensor)
        # print("B")
        action = loaded_online_net.get_action(state_tensor)
        
        # Take the action in the environment
        # print("C")
        next_state, reward, terminated, truncated,_ = env.step(action)
        score += reward
        done = terminated or truncated
        print('\r score: {:.2f} | done: {} | action: {}'.format(
            score, done, action, ), end='')
        if done:
            state = env.reset()[0]
            score = 0
        else:
            state = next_state
        done = False
        # print("D")
        # Render the environment
        env.render()
        
        # Limit the frame rate
        # clock.tick(60)

    env.close()
    # pygame.quit()




In [18]:
play_game_with_model('train_DQN_pp_500e', load_best = True)

 score: 0.00 | done: False | action: 3